In [106]:
import numpy as np 
from math import sqrt, erf
from typing import Callable
from numpy.typing import ArrayLike
from random import random

---
# Parcial 3 2024 



##  Ejercicio 2

### Constantes

In [48]:
SAMPLES = [
    1.628, 1.352, 1.800,
    1.420, 1.594, 2.132,
    1.614, 1.924, 1.692
]

NSAMPLES = len(SAMPLES)

# Muestras ordenadas
SORT_SAMPLES = np.sort(SAMPLES)

### Estimación de los parámetros

In [ ]:
# Estimación de los parámetros
MU_HAT = round(sum(SAMPLES) / len(SAMPLES), 4)
SIGMA_HAT = round(sqrt(sum((x - MU_HAT)** 2 for x in SAMPLES) / (len(SAMPLES) - 1)), 4)
print(f"MU sombrero    -> {MU_HAT}")
print(f"SIGMA sombrero -> {SIGMA_HAT}")

MU sombrero    -> 1.684
SIGMA sombrero -> 0.2418


### Estadístico

In [56]:
# Fda
def cdf_Gaussian(x:float, mu:float, sigma:float) -> float:
    """
    Función de distribución acumulada Normal
    con parámetros μ=0, σ=1

    Args:
        x (float): Valor a acumular

    Returns:
        float: Valor acumulado hasta
    """
    # Estandarizamos
    z = (x - mu) / sigma
    return erf(z / sqrt(2.0)) / 2.0 + 0.5

In [128]:
def KS_statistic(Nsamples:int, sort_samples:ArrayLike, F:Callable[[float], float]) -> float:
    """
    Calcula el estadístico de Kolmogorov-Smirnov

    Args:
        Nsamples (int): Número de muestras
        sort_samples (float): Lista ordenada de muestras

    Returns:
        float: estadístico
    """
    F_values = F(sort_samples)
    j = np.arange(1, Nsamples + 1)
    d_plus = np.max(j / Nsamples - F_values)
    d_minus = np.max(F_values - (j - 1) / Nsamples)
    return max(d_plus, d_minus)

In [129]:
F_cda = np.vectorize(lambda x: cdf_Gaussian(x=x, mu=MU_HAT, sigma=SIGMA_HAT))
d0 = KS_statistic(Nsamples=NSAMPLES,sort_samples=SORT_SAMPLES, F=F_cda)
print(f"🧐 D estadístico:{round(D, 4)}")

🧐 D estadístico:0.1535


### Test de Kolmogorov-Smirnov

In [130]:
def KS_test(Nsim:int, Nsamples:int,
            d0: float, alpha:float):
    """
    Test de Kolmogorov-Smirnov
    Decide si rechaza o no una Hipótesis Nula

    Args:
        Nsim (int): Número de simulaciones
        d0 (float): Estadístico basado en las muestras
        Nsamples (int): Número de muestras
        alpha (float): Nivel de rechazo
    """
    p_value = 0

    for _ in range(Nsim):
        #Generamos muestras aleatorias para estimar el p_valor
        uniform_samples = np.sort([random() for _ in range(Nsamples)])
        d_sim = KS_statistic(Nsamples=Nsamples, sort_samples=uniform_samples, F=lambda x:x)
        if d_sim >= d0: 
            p_value += 1
        
    p_value = round(p_value / Nsim, 4)
    print(f"☝️ p-valor obtenido: {p_value}")

    if p_value > alpha:
        print(f"😲☝️ Como {p_value} > {alpha}:")
        print("\t 😒 No hay evidencia suficiente para rechazar Ho")
    else:
        print(f"😲☝️ Como {p_value} <= {alpha}:")
        print(f"\t 🔴 Se rechaza Ho con una confianza del {int(100 * (1-alpha))}%")

In [131]:
KS_test(Nsim=10_000, Nsamples=NSAMPLES,
        d0=d0, alpha=0.04)

☝️ p-valor obtenido: 0.9621
😲☝️ Como 0.9621 > 0.04:
	 😒 No hay evidencia suficiente para rechazar Ho


---
d) Suponer que en una de las simulaciones se obtuvieron los siguientes valores para las simulaciones: U1 = 0.23, U2 = 0.12, U3 = 0.45, U4 = 0.67, U5 = 0.01, U6 = 0.51, U7 = 0.38, U8 = 0.92 y U9 = 0.84.
¿Como resulta el valor del estad´ıstico para estos datos, en relaci´on al estad´ıstico observado?

In [ ]:
NEW_SAMPLES = [
    0.23, 0.12, 0.45,
    0.67, 0.01, 0.51,
    0.38, 0.92, 0.84
]

NEW_SAMPLES_SORTED = np.sort(NEW_SAMPLES)

new_d = KS_statistic(Nsamples=len(NEW_SAMPLES_SORTED), sort_samples=NEW_SAMPLES_SORTED, F=lambda x:x)
print(f"🧐 D estadístico:{round(D, 4)}")

🧐 D estadístico:0.1535


In [139]:
_tam_muestra = len(NEW_SAMPLES)
_muestra_uniformes = [0.23, 0.12, 0.45, 0.67, 0.01, 0.51, 0.38, 0.92, 0.84]

# Calculamos el estadistico como si estuvieramos simulando:

_muestra_uniformes_ordenada = np.sort(_muestra_uniformes)
# Calculamos el estadistico de Kolmogorov Smirnov
d_j = 0
for j, u_j in enumerate(_muestra_uniformes_ordenada):
    d_j = max(d_j, ((j+1)/_tam_muestra) - u_j, u_j - (j/_tam_muestra))

print(f"d_j: {d_j}")

d_j: 0.15666666666666662
